In [37]:
import openrouteservice as ors
import folium

#client = ors.Client(key='5b3ce3597851110001cf62480413a152b510459cabf5e54bcfb4a7a9')
#client = ors.Client(key='5b3ce3597851110001cf6248f4b857eea0554d93886b4c09e748b2bf')
client = ors.Client(key='5b3ce3597851110001cf6248df64653a9cdf4dbd96171a136e5e222e')

In [30]:
import re
import pandas as pd
import math

import numpy as np

from operator import itemgetter
from heapq import nsmallest

from itertools import chain

import time

In [31]:
df_property = pd.read_csv('../data/curated/property.csv')

property_coor = []
for index, row in df_property.iterrows():
    lat = float(re.findall(r'\[(-\d+.\d+)',str(row['coordinates']))[0])
    long = float(re.findall(r'(\d+.\d+)\]',str(row['coordinates']))[0])
    property_coor.append((long,lat))

print(len(property_coor))

df_shop = pd.read_csv('../data/curated/shop_coord.csv')

shop_coor = []
for index, row in df_shop.iterrows():
    lat = row['1']
    long = row['0']
    shop_coor.append((long,lat))
    
print(len(shop_coor))

15222
363


In [33]:
import geopy.distance

In [34]:
# find the closest 3 shopping centres for each property based on distance
# api has accessed limits so that it is difficult to track the route between each shopping centre and each property


def get_shop(index):

    # calculate distance between the property and each shopping centre
    distances = [geopy.distance.geodesic((property_coor[index][1], property_coor[index][0]), (shop_coor[j][1], shop_coor[j][0])).km for j in range(len(shop_coor))]

    # find the 3 nearest shopping centres 
    index = list(range(len(shop_coor)))
    pairs = zip(index, distances)
    result = nsmallest(3, pairs, key=itemgetter(1))
    index_list = [i[0] for i in result]

    # store the name and the coordinates of the 3 nearest shopping centres 
    name_list = [df_shop['name'][i] for i in index_list]
    coor_list = [shop_coor[i] for i in index_list]
    
    result = []
    result.append(name_list)
    result.append(coor_list)

    return result

# for each property, get corresponding closest shopping centres 
name_list = []
coor_list = []

for i in range(len(property_coor)):
    result = get_shop(i)
    name_list.append(result[0])
    coor_list.append(result[1])

print(len(name_list))

15222


In [38]:
def get_shop_ors(iter):


    flatten_list = list(chain.from_iterable(coor_list[iter*34:iter*34+34]))
    combine_coor = property_coor[iter*34:iter*34+34] + flatten_list

    if iter == len(df_property)//34:
        count = len(df_property)-iter*34
    else:
        count = 34

    matrix = client.distance_matrix(
        locations=combine_coor,
        profile='driving-car',
        metrics=['distance', 'duration'],
        validate=False,
        destinations=list(range(count,len(combine_coor))),
        sources=list(range(count)),
    )

    duration_list = []
    distance_list = []
    shop_name_list = []
    shop_coor_list = []


    for i in range(count):

        min_index = np.argmin(matrix['durations'][i][i*3:i*3+3])
        min_index_flat = min_index+i*3
        duration_list.append(matrix['durations'][i][min_index_flat])
        distance_list.append(matrix['distances'][i][min_index_flat])

        shop_name_list.append(name_list[iter*34+i][min_index])
        shop_coor_list.append(coor_list[iter*34+i][min_index])


    #print("Durations in secs: {}\n".format(matrix['durations']))
    #print("Distances in m: {}".format(matrix['distances']))

    # we only want to call the whole function once at each main iteration
    result_list = []
    result_list.append(duration_list)
    result_list.append(distance_list)
    result_list.append(shop_name_list)
    result_list.append(shop_coor_list)

    return result_list

# A total of 3500 routes is allowed for each call of api, so that we can pass 34 groups of property 
# with corresponding 3 closest schools at each time

duration_list = []
distance_list = []
shop_name_list = []
shop_coor_list = []

num_call = len(df_property)//34 + 1
for i in range(num_call):
    result = get_shop_ors(i)
    print(str(i)+' success')
    if i%40 == 0:
        time.sleep(60)
    
    duration_list.append(result[0])
    distance_list.append(result[1])
    shop_name_list.append(result[2])
    shop_coor_list.append(result[3])

print(duration_list)
print(distance_list)
print(shop_name_list)
print(shop_coor_list)

0 success
1 success
2 success
3 success
4 success
5 success
6 success
7 success
8 success
9 success
10 success
11 success
12 success
13 success
14 success
15 success
16 success
17 success
18 success
19 success
20 success
21 success
22 success
23 success
24 success
25 success
26 success
27 success
28 success
29 success
30 success
31 success
32 success
33 success
34 success
35 success
36 success
37 success
38 success
39 success
40 success
41 success
42 success
43 success
44 success
45 success
46 success
47 success
48 success
49 success
50 success
51 success
52 success
53 success
54 success
55 success
56 success
57 success
58 success
59 success
60 success
61 success
62 success
63 success
64 success
65 success
66 success
67 success
68 success
69 success
70 success
71 success
72 success
73 success
74 success
75 success
76 success
77 success
78 success
79 success
80 success
81 success
82 success
83 success
84 success
85 success
86 success
87 success
88 success
89 success
90 success
91 succes

/Users/a123/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 7th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


398 success
399 success
400 success
401 success
402 success
403 success
404 success
405 success
406 success
407 success
408 success
409 success
410 success
411 success
412 success
413 success
414 success
415 success
416 success
417 success
418 success
419 success
420 success
421 success
422 success
423 success
424 success
425 success
426 success
427 success
428 success
429 success
430 success
431 success
432 success
433 success
434 success
435 success
436 success
437 success
438 success
439 success
440 success
441 success
442 success
443 success
444 success
445 success
446 success
447 success
[[57.76, 241.71, 94.27, 76.49, 57.81, 94.36, 180.74, 89.84, 240.02, 168.56, 67.22, 163.0, 167.35, 189.72, 67.07, 110.52, 241.74, 93.72, 122.77, 148.05, 12.73, 12.73, 60.22, 168.32, 119.36, 121.7, 127.55, 123.02, 98.91, 51.48, 232.79, 95.66, 82.45, 100.53], [98.91, 100.83, 149.87, 98.91, 51.48, 100.53, 100.53, 107.77, 100.53, 143.82, 98.91, 38.47, 70.82, 128.42, 143.82, 70.82, 121.26, 57.67, 196.15

In [39]:
flatten_duration_list = list(chain.from_iterable(duration_list))
flatten_distance_list = list(chain.from_iterable(distance_list))
flatten_shop_name_list = list(chain.from_iterable(shop_name_list))
flatten_shop_coor_list = list(chain.from_iterable(shop_coor_list))

print(len(flatten_distance_list))

15222


In [40]:
df = pd.read_csv('../data/curated/property.csv')

df['driving_time_to_sc'] = flatten_duration_list
df['distance_to_sc'] = flatten_distance_list
df['closest_sc_name'] = flatten_shop_name_list
df['closest_sc_coor'] = flatten_shop_coor_list

df.to_csv('../data/curated/property_sc.csv')